In [1]:

import pandas as pd
from datetime import time
from ipyfilechooser import FileChooser
from IPython.display import display
import os

# 📌 Colonnes à supprimer
cols_to_drop = [
    'Library_Version', 'Session_type', 'Device_Number',
    'Battery_Voltage', 'Motor_Turns', 'FR'
]

# 🔧 Fonction de traitement du fichier
def process_file(file_path):
    df = pd.read_csv(file_path)

    # 🧹 Supprimer les colonnes indésirables
    df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

    # 🕒 Convertir la colonne datetime
    df['Datetime'] = pd.to_datetime(df['MM:DD:YYYY hh:mm:ss'], errors='coerce')

    # 🗓️ Extraire Date et Hour
    df['Date'] = df['Datetime'].dt.date
    df['Hour'] = df['Datetime'].dt.time

    # ☀️ Déterminer la période Day/Night
    def get_period(hour):
        if pd.isnull(hour):
            return None
        if time(7, 0, 1) <= hour <= time(19, 0, 0):
            return "Day"
        else:
            return "Night"
    
    df['Period'] = df['Hour'].apply(get_period)

    # 💾 Enregistrer dans le même dossier avec _modifié
    folder = os.path.dirname(file_path)
    base = os.path.splitext(os.path.basename(file_path))[0]
    output_file = os.path.join(folder, f"{base}_Periodic.csv")

    df.to_csv(output_file, index=False)

    # ✅ Confirmation + aperçu
    print("\n✅ Le fichier a bien été enregistré !")
    print(f"📍 Chemin : {output_file}")
    print("\n🔍 Aperçu des premières lignes du fichier modifié :")
    display(df.head())

# 📂 Fonction appelée après sélection du fichier
def update_input_file(chooser):
    global input_file_path
    input_file_path = chooser.selected
    print(f"\n📂 Fichier sélectionné : {input_file_path}")
    
    if input_file_path:
        process_file(input_file_path)

# 🎛️ Interface de sélection de fichier
fc_input = FileChooser(".", show_only_dirs=False, title="<b>Sélectionnez le fichier CSV</b>")
display(fc_input)

# 🎯 Lier la sélection au traitement
fc_input.register_callback(update_input_file)



FileChooser(path='C:\Users\AudreyHay\Documents\Carla\Visual Code\HayLabAnalysis\python', filename='', title='<…

,MM:DD:YYYY hh:mm:ss,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Datetime,Date,Hour,Period
0,4/30/2025 16:34:34,Left,Left,1,0,0,0,NaN,NaN,0.00,2025-04-30 16:34:34,2025-04-30,16:34:34,Day
1,4/30/2025 16:34:58,Pellet,Left,1,0,1,0,18.41,NaN,NaN,2025-04-30 16:34:58,2025-04-30,16:34:58,Day
2,4/30/2025 16:36:33,Right,Left,1,1,1,0,NaN,NaN,0.39,2025-04-30 16:36:33,2025-04-30,16:36:33,Day
3,4/30/2025 16:37:26,Left,Left,2,1,1,0,NaN,NaN,0.16,2025-04-30 16:37:26,2025-04-30,16:37:26,Day
4,4/30/2025 16:39:06,Pellet,Left,2,1,2,0,Timed_out,248.0,NaN,2025-04-30 16:39:06,2025-04-30,16:39:06,Day


,MM:DD:YYYY hh:mm:ss,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Datetime,Date,Hour,Period
0,4/30/2025 16:32:52,Left,Left,1,0,0,0,NaN,NaN,0.00,2025-04-30 16:32:52,2025-04-30,16:32:52,Day
1,4/30/2025 16:33:06,Pellet,Left,1,0,1,0,1.64,NaN,NaN,2025-04-30 16:33:06,2025-04-30,16:33:06,Day
2,4/30/2025 16:36:57,Right,Left,1,1,1,0,NaN,NaN,0.04,2025-04-30 16:36:57,2025-04-30,16:36:57,Day
3,4/30/2025 16:36:58,Left,Left,2,1,1,0,NaN,NaN,0.01,2025-04-30 16:36:58,2025-04-30,16:36:58,Day
4,4/30/2025 16:37:01,Pellet,Left,2,1,2,0,1.96,235.0,NaN,2025-04-30 16:37:01,2025-04-30,16:37:01,Day
